In [6]:
import ipywidgets as widgets
from ipywidgets import interact
import matplotlib.pyplot as plt
import PIL, urllib
import numpy as np
import warnings
import pickle
warnings.filterwarnings("ignore", category=UserWarning) 

In [7]:
%%javascript
document.title = "My SummerSchool App";

<IPython.core.display.Javascript object>

In [8]:
default_url = 'https://upload.wikimedia.org/wikipedia/commons/2/24/Blue_Tshirt.jpg'

In [9]:
display(widgets.HTML('''
<h1>
My Summer School App
</h1>
<p style="max-width: 500px;">
This app takes a url to a .jpg or .png image.
Converts it into a grey scale 8x8 image and passes
it through the trained model on dataset of handwritten digits.
The predicted class is than simply displayed as a text.
Type in an image url and press enter.
</p>
'''))

HTML(value='\n<h1>\nMy Summer School App\n</h1>\n<p style="max-width: 500px;">\nThis app takes a url to a .jpg…

In [12]:
@interact(mt=widgets.RadioButtons(
    options=['LogisticRegression', 'PCANaiveBayes'],
    value='LogisticRegression',
    description='Model:',
    disabled=False
))
def f(mt):
    if mt == 'LogisticRegression':
        model_path = 'pretrained_lr.model'
    else:
        model_path = 'pretrained_nb.model'

    # Load the desired model
    model = pickle.load(open(model_path, 'rb'))

    @interact(
        url=widgets.Text(
            value=default_url,
            placeholder='Type in an image URL.',
            description='Image URL:',
            disabled=False,
            continuous_update=False))
    def g(url):
        try:
            # Load and transform the image
            img_arr = PIL.Image.open(
                urllib.request.urlopen(url)).convert('L').resize((8,8))

            # Plot the transformed image
            plt.figure(figsize=(5,5))
            plt.imshow(img_arr, cmap='gray', aspect='equal')
            plt.title(f'Converted into grayscale {img_arr._size} image')
            plt.axis('off')
            plt.show()   

            # Preprocess the image data
            data = np.array(img_arr).ravel().reshape(1, -1)
            
            # Pass it through the model and display the result
            display(model.predict(data))
            
        except Exception as e:
            display(e)


interactive(children=(RadioButtons(description='Model:', options=('LogisticRegression', 'PCANaiveBayes'), valu…